In [1]:
#initialize tools

import json
import os
import pandas as pd

In [2]:
# Read the json file and check that it looks ok
df = pd.read_json('purchase_data.json')
df.head()

,Age,Gender,Item ID,Item Name,Price,SN
0,38,Male,165,Bone Crushing Silver Skewer,3.37,Aelalis34
1,21,Male,119,"Stormbringer, Dark Blade of Ending Misery",2.32,Eolo46
2,34,Male,174,Primitive Blade,2.46,Assastnya25
3,21,Male,92,Final Critic,1.36,Pheusrical25
4,23,Male,63,Stormfury Mace,1.27,Aela59


In [3]:
#check to make sure we're not missing data

df.count()

Age          780
Gender       780
Item ID      780
Item Name    780
Price        780
SN           780
dtype: int64

In [4]:
players = df['SN'].unique()
players_unique_df = pd.DataFrame(players)
players_unique_df.head()

,0
0,Aelalis34
1,Eolo46
2,Assastnya25
3,Pheusrical25
4,Aela59


PLAYER COUNT

In [5]:
#TASK: print out the total player count

#NOTE: in each case for these things it looks like we're creating a new dataframe to examine (based on how to example file looks)
#also, we have to make sure to check for all the UNIQUE players

#Here I decided to use the .unique fucntion. This gives me an array of all the names. I turn this into an dataframe, then get the
#count of this dataframe and create a new dataframe to print the data in the requested format.

players = df['SN'].unique()
players_unique_df = pd.DataFrame(data=players)
players_df = pd.DataFrame({'Total Players': players_unique_df.count()})

#We could also print this info out using a pandas series. I kept the series code for my own use.
#players_s = pd.Series(players, index=['Total Players'])

players_df

,Total Players
0,573


In [6]:
#testing the .mean function

a_price = df['Price'].mean()
a_price = '${:,.2f}'.format(a_price)
a_price

'$2.93'

PURCHASING ANALYSIS (TOTAL)

In [7]:
#get the number of unique items, set this to a dataframe, and then get a count
#(I'm using a different method to check for uniques here as opposed to using the .unique function. To see if it works.)
#NOTE!! tested and does not work. Using this will set the index for the final output dataframe to 'Item Name'. I had to use
#the same method as for total price to get the final output index correct.

    #uniques_df = pd.DataFrame(df['Item Name'].value_counts())

    #uniques = uniques_df.count()

uniques = df['Item Name'].unique()
uniques_count_df = pd.DataFrame(data=uniques)
unique_total = uniques_count_df.count()

#set a variable for the price mean and convert to a dollar format

a_price = df['Price'].mean()
a_price = '${:,.2f}'.format(a_price)
a_price

#get the total number of purchases

total_p = df['Item Name'].count()

#get the total revenue

revenue = '${:,.2f}'.format(df['Price'].sum())

#create a dictionary containing the variables/lists we just developed

data_dict = {'Number of Unique Items': unique_total, 'Average Price': a_price, 'Number of Purchases': total_p, 
             'Total Revenue': revenue}

#create a dataframe from this dictionary

purchasing_analysis_df = pd.DataFrame(data=data_dict, columns=['Number of Unique Items', 'Average Price', 
                                                 'Number of Purchases', 'Total Revenue'])

purchasing_analysis_df

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,179,$2.93,780,"$2,286.33"


In [8]:
df_unique = df.drop_duplicates(subset='SN')
df_unique.head()

,Age,Gender,Item ID,Item Name,Price,SN
0,38,Male,165,Bone Crushing Silver Skewer,3.37,Aelalis34
1,21,Male,119,"Stormbringer, Dark Blade of Ending Misery",2.32,Eolo46
2,34,Male,174,Primitive Blade,2.46,Assastnya25
3,21,Male,92,Final Critic,1.36,Pheusrical25
4,23,Male,63,Stormfury Mace,1.27,Aela59


GENDER DEMOGRAPHICS

In [9]:
#TASK: gender demographics. Percentage and Count of Male Players, Percentage and Count of Female Players
#Percentage and Count of Other / Non-Disclosed

genders = df_unique['Gender'].value_counts()
genders_df = pd.DataFrame(data=genders)
genders_df

,Gender
Male,465
Female,100
Other / Non-Disclosed,8


In [10]:
#calculate the percentage for each gender
genders_df['Percentage of Players'] = 100 * (genders_df['Gender'] / genders_df['Gender'].sum())

#rename column and reorder
genders_df = genders_df.rename(columns={'Gender': 'Total Count'})
genders_df_reorder = genders_df[['Percentage of Players', 'Total Count']]
genders_df_reorder

,Percentage of Players,Total Count
Male,81.151832,465
Female,17.452007,100
Other / Non-Disclosed,1.396161,8


PURCHASING ANALYSIS (GENDER)

In [11]:
gender_purchasing = df.groupby(['Gender'])
g_count = pd.DataFrame(gender_purchasing.count()['Item ID'])
g_average = pd.DataFrame(gender_purchasing.mean()['Price'])
g_total = pd.DataFrame(gender_purchasing.sum()['Price'])

#reset the indices to allow the merge to happen
g_count.reset_index(inplace=True)
g_average.reset_index(inplace=True)
g_total.reset_index(inplace=True)
g_total.columns = ['Gender', 'Total Purchase Value']

#merge the dataframes
p_analysis_df = pd.merge(g_count, g_average, on='Gender', how='outer')
p_analysis_df.columns = ['Gender', 'Purchase Count', 'Average Purchase Price']
p_analysis_df = pd.merge(p_analysis_df, g_total, on='Gender', how='outer')

p_analysis_df = p_analysis_df.sort_values(['Purchase Count'], ascending=False)
p_analysis_df = p_analysis_df.set_index('Gender')

#add the normalized totals to the dataframe

p_analysis_df['Normalized Totals'] = p_analysis_df['Total Purchase Value'] / genders_df['Total Count']

#change the display format to dollars
pd.options.display.float_format = '${:,.2f}'.format

p_analysis_df

,Purchase Count,Average Purchase Price,Total Purchase Value,Normalized Totals
Gender,,,,
Male,633,$2.95,"$1,867.68",$4.02
Female,136,$2.82,$382.91,$3.83
Other / Non-Disclosed,11,$3.25,$35.74,$4.47


AGE DEMOGRPHICS

In [12]:
#create bins and sort data based on age

bins = [0, 9, 14, 19, 24, 29, 34, 39, 100]

age_names = ['>10', '10-14', '15-19', '20-24', '25-29', '30-34', '35-39', '>40']

age_column = pd.cut(df_unique['Age'], bins, labels=age_names)

age_counts = age_column.value_counts()
age_counts_df = pd.DataFrame(age_counts, index=age_names)
age_counts_df['Percentage of Players'] = 100 * age_counts_df['Age'] / age_counts_df['Age'].sum()
age_counts_df = age_counts_df.rename(columns={'Age': 'Total Count'})

#change the numeric display to a percent
pd.options.display.float_format = '{:,.2f}%'.format
age_counts_reorder = age_counts_df[['Percentage of Players', 'Total Count']]
age_counts_reorder

,Percentage of Players,Total Count
>10,3.32%,19
10-14,4.01%,23
15-19,17.45%,100
20-24,45.20%,259
25-29,15.18%,87
30-34,8.20%,47
35-39,4.71%,27
>40,1.92%,11


PURCHASING ANALYSIS (AGE)

In [51]:
#get the total purchases in each age bin (do not use the unique names for this, use the total)
age_purchases = pd.cut(df['Age'], bins, labels=age_names)
df['Age Purchases'] = age_purchases

#chagne to dollar format for the display output
pd.options.display.float_format = '${:,.2f}'.format

age_groups = df.groupby(['Age Purchases'])
total_p_value = age_groups['Price'].sum()
total_p_value

mean_purchase = age_groups['Price'].mean()

#construct the dataframe by age bin and add the revenue columns to it
purchase_age_counts = pd.DataFrame(age_purchases.value_counts())
purchase_age_counts

purchase_age_counts['Average Purchase Price'] = mean_purchase
purchase_age_counts['Total Purchase Value'] = total_p_value

#reorder the index of the dataframe and rename the Age column
purchase_age_counts = purchase_age_counts.sort_index(axis=0)
purchase_age_counts = purchase_age_counts.rename(columns={'Age': 'Purchase Count'})

#get the normalized totals
purchase_age_counts['Normalized Totals'] = purchase_age_counts['Total Purchase Value'] / age_counts_reorder['Total Count']

purchase_age_counts

,Purchase Count,Average Purchase Price,Total Purchase Value,Normalized Totals
>10,28,$2.98,$83.46,$4.39
10-14,35,$2.77,$96.95,$4.22
15-19,133,$2.91,$386.42,$3.86
20-24,336,$2.91,$978.77,$3.78
25-29,125,$2.96,$370.33,$4.26
30-34,64,$3.08,$197.25,$4.20
35-39,42,$2.84,$119.40,$4.42
>40,17,$3.16,$53.75,$4.89


TOP SPENDERS

In [82]:
#group the purchase data by SN and get the sums spent

spenders = df.groupby(['SN'])
top_spenders_sum = spenders['Price'].sum()
top_spenders_df = pd.DataFrame(top_spenders_sum)

top_spenders_number = df['SN'].value_counts()
top_spenders_df['Purchase Count'] = top_spenders_number

top_spenders_df = top_spenders_df.sort_values('Purchase Count', ascending=False)
top_spenders_df['Average Purchase Price'] = top_spenders_df['Price'] / top_spenders_df['Purchase Count']

top_spenders_df = top_spenders_df.rename(columns={'Price': 'Total Purchase Value'})
top_spenders_df = top_spenders_df[['Purchase Count', 'Average Purchase Price', 'Total Purchase Value']]

top_spenders_df.head()

,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Undirrala66,5,$3.41,$17.06
Mindimnya67,4,$3.18,$12.74
Qarwen67,4,$2.49,$9.97
Saedue76,4,$3.39,$13.56
Sondastan54,4,$2.56,$10.24


MOST POPULAR ITEMS